In [14]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from PIL import Image

# Set image dimensions
IMG_HEIGHT = 64
IMG_WIDTH = 64
CHANNELS = 3

# Load and preprocess dataset
def load_image_data(data_dir):
    images = []
    labels = []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        for image_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_name)
            img = Image.open(img_path).resize((IMG_WIDTH, IMG_HEIGHT))
            img = np.array(img) / 255.0  # Normalize
            images.append(img)
            labels.append(0 if class_name == 'good' else 1)  # 0 for good, 1 for defect
    return np.array(images), np.array(labels)

# Example usage
data_dir = r"C:\Users\Admin\Downloads\dataset1" # Update with your dataset path
images, labels = load_image_data(data_dir)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Generator model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_dim=100),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        layers.Dense(1024),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        layers.Dense(np.prod((IMG_HEIGHT, IMG_WIDTH, CHANNELS)), activation='tanh'),
        layers.Reshape((IMG_HEIGHT, IMG_WIDTH, CHANNELS))
    ])
    return model

# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Build the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = layers.Input(shape=(100,))
    generated_img = generator(gan_input)
    gan_output = discriminator(generated_img)
    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')
    return gan

# Compile models
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gan = build_gan(generator, discriminator)

# Training the GAN
def train_gan(generator, discriminator, gan, epochs, batch_size):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, train_images.shape[0], half_batch)
        real_imgs = train_images[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.array([1] * batch_size)  # Label "1" for fooling discriminator
        g_loss = gan.train_on_batch(noise, valid_y)
        
        # Print progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")

# Run training
train_gan(generator, discriminator, gan, epochs=1000, batch_size=32)



1/1 [==============================] - 0s 89ms/step
0 [D loss: 0.6793673038482666, acc.: 53.125%] [G loss: 1.232141375541687]
1/1 [==============================] - 0s 20ms/step
100 [D loss: 0.027553340485610534, acc.: 100.0%] [G loss: 40.24558639526367]
1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 18ms/step
200 [D loss: 0.004240113310515881, acc.: 100.0%] [G loss: 269.05255126953125]
1/1 [==============================] - 0s 20ms/step
300 [D loss: 0.026140228903386742, acc.: 100.0%] [G loss: 71.27106475830078]
1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step
400 [D loss: 0.00035172584466636186, acc.: 100.0%] [G loss: 213.6285400390625]
1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 22ms/step
500 [D loss: 0.03996531665325165, acc.: 100.0%] [G loss: 696.7283935546875]
1/1 [==============================] - 0s 19ms/step
600 [D loss: 0.00024364951241295785, acc.: 100.0%] [G loss: 663.3388671875]
1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step
700 [D loss: 0.0007188561139628291, acc.: 100.0%] [G loss: 254.62896728515625]
1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 23ms/step
800 [D loss: 0.005416600964964791, acc.: 100.0%] [G loss: 225.48095703125]
1/1 [==============================] - 0s 20ms/step
900 [D loss: 0.00198501069098711, acc.: 100.0%] [G loss: 196.01052856445312]
1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 18ms/step


#  Anomaly Detection:

In [17]:
def calculate_anomaly_score(generator, real_images):
    noise = np.random.normal(0, 1, (real_images.shape[0], 100))
    generated_images = generator.predict(noise)
    
    # Calculate L2 distance (reconstruction error)
    loss = np.mean(np.square(real_images - generated_images), axis=(1, 2, 3))
    
    return loss

# Get anomaly score for test images
anomaly_scores = calculate_anomaly_score(generator, test_images)

# Set threshold for classifying images as defective
threshold = 0.1  # You can tune this value
predicted_labels = (anomaly_scores > threshold).astype(int)


1/1 [==============================] - 0s 18ms/step


# Evaluation:


In [18]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the binary classification
accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(classification_report(test_labels, predicted_labels))


Accuracy: 42.86%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.43      1.00      0.60         6

    accuracy                           0.43        14
   macro avg       0.21      0.50      0.30        14
weighted avg       0.18      0.43      0.26        14



C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
new_image_dir = r"C:\Users\Admin\Desktop\New folder" # Path to your new images
new_images, _ = load_image_data(new_image_dir)  # Using the same loading function, but without labels


In [24]:
# Assuming the generator model has been trained
def generate_images(generator, num_images):
    noise = np.random.normal(0, 1, (num_images, 100))  # Random noise input
    generated_images = generator.predict(noise)
    return generated_images

# Generate images for your new test data
generated_images = generate_images(generator, new_images.shape[0])


1/1 [==============================] - 0s 19ms/step


In [25]:
def calculate_anomaly_score(real_images, generated_images):
    # Calculate reconstruction error (L2 distance)
    reconstruction_error = np.mean(np.square(real_images - generated_images), axis=(1, 2, 3))
    return reconstruction_error

# Get the anomaly score for the new images
anomaly_scores = calculate_anomaly_score(new_images, generated_images)


In [26]:
threshold = 0.9 # You need to adjust this based on your dataset's performance
predicted_labels = (anomaly_scores > threshold).astype(int)  # 0 for good, 1 for defect

# Print the results for each image
for i, score in enumerate(anomaly_scores):
    print(f"Image {i+1}: Anomaly Score = {score}, Predicted: {'Defect' if predicted_labels[i] == 1 else 'Good'}")


Image 1: Anomaly Score = 0.9362883843280239, Predicted: Defect
Image 2: Anomaly Score = 0.9686184228575501, Predicted: Defect
Image 3: Anomaly Score = 0.954803806236218, Predicted: Defect
Image 4: Anomaly Score = 0.9599442108090027, Predicted: Defect
Image 5: Anomaly Score = 0.9723107630710711, Predicted: Defect
Image 6: Anomaly Score = 0.9545207611927796, Predicted: Defect


In [ ]:
pip uninstall scikit-learn imbalanced-learn

In [49]:
pip install --upgrade scikit-learn


Note: you may need to restart the kernel to use updated packages.


# new

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from imblearn.over_sampling import SMOTE
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Set image parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
CHANNELS = 3

# 1. Load and Preprocess Images
def load_image_data(data_dir):
    images = []
    labels = []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        label = 0 if class_name == "good" else 1  # 0: good, 1: defect
        for image_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_name)
            img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = image.img_to_array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

# Example usage
data_dir = r"C:\Users\Admin\Downloads\dataset"  # Your dataset directory
images, labels = load_image_data(data_dir)

# Normalize images
images = images / 255.0

# 2. Feature Extraction Using Pre-trained CNN (ResNet50)
feature_extractor = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))

def preprocess_images(images):
    return preprocess_input(images)

def extract_features(model, images):
    preprocessed_images = preprocess_images(images)
    features = model.predict(preprocessed_images)
    return features

features = extract_features(feature_extractor, images)

# 3. Apply SMOTE to Balance the Dataset
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = smote.fit_resample(features, labels)

# 4. Define the GAN Model (Generator and Discriminator)
# Generator Model
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(IMG_HEIGHT * IMG_WIDTH * CHANNELS, activation='tanh'))
    model.add(Reshape((IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
    return model

# Discriminator Model
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
    model.add(Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and Compile Discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build Generator
generator = build_generator()

# GAN Model: Generator -> Discriminator
z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)

gan = Model(z, valid)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# 5. Train the GAN
def train_gan(generator, discriminator, gan, epochs, batch_size=32):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, X_resampled.shape[0], batch_size)
        real_images = X_resampled[idx].reshape(-1, IMG_HEIGHT, IMG_WIDTH, CHANNELS)

        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_images, valid)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, valid)

        # Print progress
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}%] [G loss: {g_loss}]")

# Train the model
train_gan(generator, discriminator, gan, epochs=100, batch_size=32)

# 6. Test the GAN on New Images
def test_gan_on_new_images(generator, test_images):
    noise = np.random.normal(0, 1, (test_images.shape[0], 100))
    generated_images = generator.predict(noise)

    # Visualization of real vs generated images
    visualize_results(test_images, generated_images)

def visualize_results(real_images, generated_images, num_images=5):
    for i in range(num_images):
        plt.subplot(2, num_images, i + 1)
        plt.imshow(real_images[i])
        plt.title("Original")
        plt.axis('off')

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(generated_images[i])
        plt.title("Generated")
        plt.axis('off')

    plt.show()

# Test on some new data
new_images, _ = load_image_data(r"C:\Users\Admin\Desktop\New folder\test_def")
test_gan_on_new_images(generator, new_images)





3/3 [==============================] - 1s 201ms/step


ValueError: cannot reshape array of size 65536 into shape (128,128,3)

In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from imblearn.over_sampling import SMOTE
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Define image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128
CHANNELS = 3

# Load and preprocess a single image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize image
    return img_array

# Load images from a directory
dataset_dir = r"C:\Users\Admin\Downloads\dataset\good"
image_paths = [os.path.join(dataset_dir, fname) for fname in os.listdir(dataset_dir) if fname.endswith('.jpg') or fname.endswith('.png')]
images = np.array([preprocess_image(path) for path in image_paths])

print(f"Loaded {len(images)} images with shape: {images.shape}")

# Check that the number of images in X_resampled and their dimensions are correct
expected_size_per_image = IMG_HEIGHT * IMG_WIDTH * CHANNELS  # 128 * 128 * 3 = 49152
total_images = X_resampled.size // expected_size_per_image  # Ensure the size matches

if X_resampled.size % expected_size_per_image != 0:
    raise ValueError(f"Inconsistent size of X_resampled: expected multiple of {expected_size_per_image}")

# Reshape correctly if total number of elements matches the expected number of images
X_resampled = X_resampled.reshape(50, IMG_HEIGHT, IMG_WIDTH, CHANNELS)
print(f"Number of images: {len(X_resampled)}")
print(f"Number of labels: {len(labels)}")
labels = labels[:len(X_resampled)]  # Truncate labels to match the number of images
print(f"Shape of X_resampled before reshape: {X_resampled.shape}")

X_resampled, y_resampled = smote.fit_resample(features.reshape(len(features), -1), labels)





# Define the feature extractor
feature_extractor = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))

# Feature extraction function
def extract_features(model, images):
    preprocessed_images = preprocess_input(images)  # Preprocess for ResNet50
    features = model.predict(preprocessed_images)
    return features

features = extract_features(feature_extractor, images)

# Apply SMOTE for balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_resampled, y_resampled = smote.fit_resample(features.reshape(len(features), -1), labels)

# 4. Define the GAN Model (Generator and Discriminator)
# Generator Model
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(IMG_HEIGHT * IMG_WIDTH * CHANNELS, activation='tanh'))
    model.add(Reshape((IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
    return model

# Discriminator Model
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
    model.add(Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and Compile Discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build Generator
generator = build_generator()

# GAN Model: Generator -> Discriminator
z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)

gan = Model(z, valid)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# 5. Train the GAN
def train_gan(generator, discriminator, gan, epochs, batch_size=32):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, X_resampled.shape[0], batch_size)
        real_images = X_resampled[idx].reshape(-1, IMG_HEIGHT, IMG_WIDTH, CHANNELS)

        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_images, valid)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, valid)

        # Print progress
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}%] [G loss: {g_loss}]")

# Train the model
train_gan(generator, discriminator, gan, epochs=100, batch_size=32)

# 6. Test the GAN on New Images
def test_gan_on_new_images(generator, test_images):
    noise = np.random.normal(0, 1, (test_images.shape[0], 100))
    generated_images = generator.predict(noise)

    # Visualization of real vs generated images
    visualize_results(test_images, generated_images)

def visualize_results(real_images, generated_images, num_images=5):
    for i in range(num_images):
        plt.subplot(2, num_images, i + 1)
        plt.imshow(real_images[i])
        plt.title("Original")
        plt.axis('off')

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(generated_images[i])
        plt.title("Generated")
        plt.axis('off')

    plt.show()

# Test on some new data
new_images, _ = load_image_data(r"C:\Users\Admin\Desktop\New folder\test_def")
test_gan_on_new_images(generator, new_images)


Loaded 50 images with shape: (50, 128, 128, 3)


NameError: name 'X_resampled' is not defined

# Transformers